In [2]:
import h5py
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
from glob import glob
from tqdm import tqdm

In [3]:
h5_base = "/data/"
jpg_dest_folder = "/data/all_cxr_jpg/"
if not os.path.exists(jpg_dest_folder):
    os.makedirs(jpg_dest_folder)
h5_files = glob(h5_base + '/*.hdf5')

In [6]:
deid_keys_df = pd.read_csv("/data/UCSD_cxr/deid-keys.csv")
deid_keys_df = deid_keys_df.drop_duplicates()
deid_keys_df.head()

,Study Key,Accession Number,Phonetic ID
0,2.25.202502464261767421468027007730650464986,54173096,Jahire
1,2.25.220161368548769855356092496684218090572,54316630,Behejee
2,2.25.105970840676040350164955593094376386679,54166055,Kureyil
3,2.25.60530620917870643800767150797459760050,54309743,Isbritem
4,2.25.108489539071837725029506164517778651174,54296790,Queroblo


In [9]:
def get_accession_ids(h5_keys):
    imgs_keys = pd.DataFrame([key.split("__") for key in h5_keys], columns=["Phonetic ID", "Study Key", "Photo_ID"])
    first_image = imgs_keys[imgs_keys["Photo_ID"] == "0"]
    merge_study_key = first_image.merge(deid_keys_df[["Study Key", "Accession Number"]], left_on="Study Key", right_on="Study Key")
    return merge_study_key

In [10]:
def convert_h5_to_jpg(image_data, jpg_file_path):
    image_data = ((image_data - image_data.min()) / (image_data.ptp()) * 255).astype(np.uint8)
    img = Image.fromarray(image_data)
    img.save(jpg_file_path)

In [8]:
for f_name in tqdm(h5_files):
    h5_data = h5py.File(f_name)
    keys = list(h5_data.keys())
    merged_df = get_accession_ids(keys)
    for i, (p_id, s_id, photo_id, acc_num) in merged_df.iterrows():
        dataset = h5_data[f"{p_id}__{s_id}__{photo_id}"][:]
        jpg_file_path = os.path.join(jpg_dest_folder, f'{p_id}_{acc_num}.jpg')
        convert_h5_to_jpg(dataset, jpg_file_path)

 40%|████      | 2/5 [30:35<45:49, 916.53s/it]  /tmp/ipykernel_218059/2928529968.py:2: RuntimeWarning: invalid value encountered in divide
  image_data = ((image_data - image_data.min()) / (image_data.ptp()) * 255).astype(np.uint8)
/tmp/ipykernel_218059/2928529968.py:2: RuntimeWarning: invalid value encountered in cast
  image_data = ((image_data - image_data.min()) / (image_data.ptp()) * 255).astype(np.uint8)
 80%|████████  | 4/5 [58:48<14:23, 863.02s/it]